# A sample research
- Whether in Each region, teams of Big 4 [NHL,NFl, NBA, MLB] has the same `WIN_LOSS-Ratio` or Not

- **Used T-statistic though*

- **Emphesized on The flexbility of programming, which would be quite `onerous` in Excel** like **string manipulation**

## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

In [1]:
import pandas as pd
import numpy as np
import scipy as stats
from scipy.stats import ttest_ind

In [108]:
df = pd.read_html("assets/wikipedia_data.html")[1]
df.head();

In [109]:
df = df.rename(columns={"Population (2016 est.)[8]": "Population"})
df.head()

,Metropolitan area,Country,Pop.rank,Population,B4,NFL,MLB,NBA,NHL,B6,MLS,CFL
0,New York City,United States,1,20153634,9,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils[note 3],11,Red BullsNew York City FC,—
1,Los Angeles,United States,2,13310447,8,RamsChargers[note 4],DodgersAngels,LakersClippers,KingsDucks,10,GalaxyLos Angeles FC[note 5],—
2,San Francisco Bay Area,United States,6,6657982,6,49ersRaiders[note 6],GiantsAthletics,Warriors,Sharks[note 7],7,Earthquakes,—
3,Chicago,United States,3,9512999,5,Bears[note 8],CubsWhite Sox,Bulls[note 9],Blackhawks,6,Fire,—
4,Dallas–Fort Worth,United States,4,7233323,4,Cowboys,Rangers,Mavericks,Stars,5,FC Dallas,—


In [110]:
# lets take the necessery columns:
df = df[["Metropolitan area", "Population", "NFL", "NHL", "MLB", "NBA"]]
df.head()

,Metropolitan area,Population,NFL,NHL,MLB,NBA
0,New York City,20153634,GiantsJets[note 1],RangersIslandersDevils[note 3],YankeesMets[note 2],KnicksNets
1,Los Angeles,13310447,RamsChargers[note 4],KingsDucks,DodgersAngels,LakersClippers
2,San Francisco Bay Area,6657982,49ersRaiders[note 6],Sharks[note 7],GiantsAthletics,Warriors
3,Chicago,9512999,Bears[note 8],Blackhawks,CubsWhite Sox,Bulls[note 9]
4,Dallas–Fort Worth,7233323,Cowboys,Stars,Rangers,Mavericks


##### Chuck of the dirt in the data that needed to be clean
***some example in `NHL` column would be***

**These are same in other columns that needed to be clean**

In [111]:
### Now clean the NFL columns
import re
[x for x in df['NHL'] if re.search("[^A-Za-z]+", x)]
# These are the Chunck of dirt taht i have to clean!

['RangersIslandersDevils[note 3]',
 'Sharks[note 7]',
 'Flyers[note 13]',
 'Wild[note 16]',
 'Avalanche[note 18]',
 'Red Wings',
 'Maple Leafs',
 '—',
 '[note 25]',
 'Penguins[note 28]',
 '[note 32]',
 '[note 34]',
 '—',
 '[note 39]',
 'Blues[note 43]',
 '—',
 '—',
 '—',
 '—',
 '—',
 '—',
 '—',
 'Blue Jackets',
 '—',
 'Jets[note 61]',
 '—',
 '—',
 '—',
 'Golden Knights',
 '—',
 '—',
 '—',
 '—',
 '[note 71]',
 '—',
 '31']

In [206]:
# lets see in others too
[x for x in df['NFL'] if re.search("[^A-Za-z]+", x)];
# assume all have the same problems


In [113]:
## Clean the whole set of columns with the loop

# replace the chunk with the ""
for clm in df.iloc[:, 2:].columns:
    df[clm] = df[clm].replace({"[^A-Za-z].*": ""}, regex=True)
[x for x in df['NHL'] if re.search("[^A-Za-z]+", x)];

#### NHL Dataframe


In [204]:
## Lets import the NHL df
nhl_df = pd.read_csv("assets/nhl.csv")
nhl_df.head();


In [115]:
# lets clean the team columns
#[x for x in nhl_df['team'] if re.search("[^A-Za-z ]+", x)]
nhl_df['team'] = nhl_df['team'].replace({"[^A-Za-z. ]*": ""}, regex= True)
[x for x in nhl_df['team'] if re.search("[^A-Za-z ]+", x)];

# which is fine

In [116]:
nhl_df.shape;

In [117]:
[x for x in nhl_df['team'] if "St" in x];

In [120]:
# make a function to make that a bit easier

def clean(sport):
    sport["team"] = sport['team'].replace({"[^A-Za-z. ]*": ""}, regex=True)
    
    return sport[sport['year'] == 2018]
nfl_df = pd.read_csv("assets/nfl.csv")
nfl_df = clean(nfl_df);
    

In [121]:
# make a funtion to get the region matching with df['Metropolitian area']

def giving_reg(sport_df, reg_df):
    
    #assign the team name to reg colum
    sport_df['reg'] = sport_df['team']
    
    # get the reg
    for reg in reg_df['Metropolitan area']:
        # get the each team
        for team in sport_df['reg']:
            #condition whether the team name starts with first 5 words of reg_df
            if not reg.startswith(team[:5]): continue
            sport_df['reg'] = sport_df['reg'].replace(to_replace=team , value=reg)
    return sport_df


nfl_df = giving_reg(nfl_df, df)
nfl_df.head();

In [7]:
# lets make a fuction which detectes such assymetry
def assymetry(sport, df):
    lis = []
    for reg in sport["reg"].unique():
        
        if reg not in list(df['Metropolitan area'].unique()):
            lis.append(reg)
    return lis

In [182]:
# need to hard code to replace the reg that doesn't match with [reg] of df
nhl_df['reg'] = nhl_df['reg'].replace({"St. Louis Blues" : "St. Louis",
                   "Florida Panthers": "Miami–Fort Lauderdale",
                   'Carolina Hurricanes' : 'Raleigh',
                   "San Jose Sharks" : 'San Francisco Bay Area',
                   'New Jersey Devils': "New York City",
                   'Colorado Avalanche': "Denver",
                   "Vegas Golden Knights" : "Las Vegas",
                   "Arizona Coyotes": "Phoenix",
                    "Anaheim Ducks": "Los Angeles"})

In [136]:
[x for x in nhl_df['reg'] if "" in x]
nhl_df[nhl_df['team'] == "St. Louis Blues"][['team', "reg"]];

In [137]:
## Finish the Nhl_clean
## nfl;

In [138]:
nfl_df.head();

In [139]:
assymetry(nfl_df, df);

In [140]:
nfl_df['reg'] = nfl_df['reg'].replace({"Arizona Cardinals": "Phoenix",
                                "Carolina Panthers": "Charlotte",
                                "New England Patriots": "Boston",
                                 "Oakland Raiders":  "San Francisco Bay Area",
                                "Tennessee Titans" : "Nashville"});

In [141]:
[x for x in nfl_df['reg'] if "Char" in x];

In [143]:
## Which is fine
nba_df.head();



In [144]:
assymetry(nba_df, df);

In [145]:
nba_df['reg'] = nba_df['reg'].replace({"Golden State Warriors": "San Francisco Bay Area",
                                "(Utah Jazz": "Salt Lake City",
                                "Brooklyn Nets": "New York City"})
nba_df[nba_df['team'] == "Golden State Warriors"][["team", "reg"]];

In [146]:
# find the team that doesn't have the name resembling the region
mlb_df = giving_reg(mlb_df, df)
assymetry(mlb_df, df);

In [147]:
# lets hardcoded it
mlb_df['reg'] = mlb_df['reg'].replace({"Oakland Athletics" : "San Francisco Bay Area",
                                    "Texas Rangers"  : "Dallas–Fort Worth",
                                    "Colorado Rockies": "Denver",
                                    "Arizona Diamondbacks" : "Phoenix"})
mlb_df[mlb_df['reg'] == 'St. Louis'];

In [153]:
df.columns;

In [154]:
# lets check for final
mlb_df[mlb_df['reg'] == "St. Louis"][['team', "reg"]];

#### show the sample of NHL (National Hokey League) Intially

In [175]:
nhl = pd.read_csv("assets/nhl.csv") 
nhl.head()

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League
0,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,2018,NHL
1,Tampa Bay Lightning*,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL
2,Boston Bruins*,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL
3,Toronto Maple Leafs*,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL
4,Florida Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL


There are more of **team names** with some sort incoherence 
like after each name, an **astrick** sign , which really make the data not usefull in analysis
let's see an example of 4 such name

In [176]:
[x for x in nhl['team'] if re.search("[^A-Za-z. ]+", x)][:4]

['Tampa Bay Lightning*',
 'Boston Bruins*',
 'Toronto Maple Leafs*',
 'Washington Capitals*']

defining a function for clean the incoherence in the name,and make it common
### basic sport_df clean fun

after running the function, customised for these type of datasets, **see there are no abnomality in names**


In [177]:
def clean(sport):
    sport["team"] = sport['team'].replace({"[^A-Za-z. ]*": ""}, regex=True)
    
    return sport[sport['year'] == 2018]
nhl = clean(nhl)
[x for x in nhl['team'] if re.search("[A-Za-z. ]+", x)][:4]

['Atlantic Division',
 'Tampa Bay Lightning',
 'Boston Bruins',
 'Toronto Maple Leafs']

### Assigning Region in each datasets

Look there was no region for each team!
**Here comes the action of** ***Python Programming***, **because in Excel, it almost quite tormenting to assign region** where *region needed to be matched from a genrealized data frame* which I called **root dataframe**

Not a usual match
Assymetry:
- 1 - There are some names of teams which matched in some portion with the name of region like, ***team: St. louis Cadaver or sth*** and ***region: St. louis***
- meaning there are names which portionately resemble the name of region
- there are other names which completely different from its region ***like Dhaka : tigar cricket team*** which not really resemble the name a bit
- so I needed to hard codded regions names of those team's name, reference from the root dataframe

* Define a function `giving region` that do the task for me in each sport_datasets

**look at the sample of the data, after running** ***giving_reg function***

In [178]:
giving_reg(nhl, df).head()

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League,reg
0,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,2018,NHL,Atlanta
1,Tampa Bay Lightning,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL,Tampa Bay Area
2,Boston Bruins,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL,Boston
3,Toronto Maple Leafs,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL,Toronto
4,Florida Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL,Florida Panthers


### giving_reg function


In [179]:
def giving_reg(sport_df, reg_df):
    
    #assign the team name to reg colum
    sport_df['reg'] = sport_df['team']
    
    # get the reg
    for reg in reg_df['Metropolitan area']:
        # get the each team
        for team in sport_df['reg']:
            #condition whether the team name starts with first 5 words of reg_df
            if not reg.startswith(team[:5]): continue
            sport_df['reg'] = sport_df['reg'].replace(to_replace=team , value=reg)
    return sport_df

## Assymetry (optional func)

After Giving the region in each team, some teams region needed to hard coded
- **how do i find the names of teams whose doesn't resemble the region at all**
- A small `optional Function` called `assymetry`, customised for these specific problem, made to find out the the team's name which has inconsistent with the region name
- for instance, in `NHL` dataframe, these team's name doesn't match with some portion of thier region name
- for example: **Florida Partner** in **Miami region**

In [180]:
assymetry(nhl,df)

['Florida Panthers',
 'Metropolitan Division',
 'New Jersey Devils',
 'Carolina Hurricanes',
 'Central Division',
 'Colorado Avalanche',
 'Pacific Division',
 'Vegas Golden Knights',
 'Anaheim Ducks',
 'San Jose Sharks',
 'Arizona Coyotes']

In [181]:
def assymetry(sport, df):
    lis = []
    for reg in sport["reg"].unique():
        
        if reg not in list(df['Metropolitan area'].unique()):
            lis.append(reg)
    return lis

## merger fuction


After that, i need `parameter of interests` or required `columns` to make necessery analysis

for instance, i need to have the `population` columns which is in the `root dataframe`

so Each Sports **Dataframe** needed to be merged with **root Dataframe** on the common columns

which is `reg` in sports columns and `Metropolian area` in **Root Dataframe**

a customised function `merger` that do the works for me in each sport datasets

**let's look at a sample of it**
 

In [182]:
nhl = merger(nhl, df).head()
nhl.head()

,reg,Population,W,L
0,New York City,20153634,35,37
1,New York City,20153634,34,39
2,Los Angeles,13310447,45,29
3,Chicago,9512999,33,39
4,Dallas–Fort Worth,7233323,42,32


In [17]:
def merger(sport_df, df):
    result = (df.merge(sport_df, how="inner", left_on="Metropolitan area", right_on="reg")
                                    .loc[:, ['reg', "Population", "W", "L"]])
    return result

### Function for to_numeric


A function that convert the `object` *datatype* into `Integer` datatype
let's call it `mk_numeric`
let's look at a sample of it

In [183]:
nhl = mk_numeric(nhl)
nhl.dtypes

reg           object
Population     int64
W              int64
L              int64
dtype: object

In [13]:
def mk_numeric(sport_df):
    # iterate over the column name
    for clm in sport_df.columns:
        if clm =="reg" : continue
        sport_df[clm] = pd.to_numeric(sport_df[clm], errors = 'coerce')
    return sport_df


### Function for win_loss ratio

To count the each region's win-loss-ratio

`win_loss_ratio` function giving the whole dataset with the calculated `win-loos-ratio`

a sample of it

In [184]:
nhl = win_loss_ratio(nhl)
nhl.head()

,reg,Population,W,L,W_L_ratio
0,Chicago,9512999,33,39,0.458333
1,Dallas–Fort Worth,7233323,42,32,0.567568
2,Los Angeles,13310447,45,29,0.608108
3,New York City,20153634,69,76,0.475862


In [29]:
def win_loss_ratio(sport_df):
    sport_df = sport_df.groupby(["reg", "Population"]).agg({"W": np.sum,
                                                "L" : np.sum}).reset_index()
    
    sport_df["W_L_ratio"] = sport_df["W"] / (sport_df["W"] + sport_df['L'])
    
    return sport_df.fillna(0)

### Function for correlation


A function corr, customized for giving the **correaltion** of each `final clean datasets` between the , let's say `Population` and `win-loss-ratio`

In [15]:
import scipy.stats as stats
def corr(sport_df):
    return stats.pearsonr(sport_df['Population'], sport_df['W_L_ratio'])

### NHL (national Hocky Datasets) 
after **making** these `function`: 

let's find the `correlation` between `Population` in **each region** and thier `win loss ratio` in respective **sports**

In [187]:
nhl_df = pd.read_csv("assets/nhl.csv")
nhl_df = clean(nhl_df)
nhl_df = giving_reg(nhl_df, df)
nhl_df['reg'] = nhl_df['reg'] = nhl_df['reg'].replace({"St. Louis Blues" : "St. Louis",
                   "Florida Panthers": "Miami–Fort Lauderdale",
                   'Carolina Hurricanes' : 'Raleigh',
                   "San Jose Sharks" : 'San Francisco Bay Area',
                   'New Jersey Devils': "New York City",
                   'Colorado Avalanche': "Denver",
                   "Vegas Golden Knights" : "Las Vegas",
                   "Arizona Coyotes": "Phoenix",
                    "Anaheim Ducks": "Los Angeles"})

# nhl_df = (df.merge(nhl_df, how="inner", left_on="Metropolitan area", right_on="reg")
#                                     .loc[:, ['reg', "Population", "W", "L"]])
nhl_df = merger(nhl_df, df)

# for clm in nhl_df.columns:
#     if clm == "reg": continue
#     nhl_df[clm] = pd.to_numeric(nhl_df[clm], errors="coerce")
nhl_df = mk_numeric(nhl_df)
nhl_df.dtypes
nhl_df = win_loss_ratio(nhl_df)
nhl_df.head()
nhl_df = nhl_df.iloc[1:, :] # ignore the Atlanta columns which doesn't have value in sports DF
print("Correlation: " +str(corr(nhl_df)[0])+ " P_value: " +str(corr(nhl_df)[1])) 

Correlation: 0.012308996455744264 P_value: 0.9504308637909502


In [188]:
nhl_df.shape;

In [189]:
#nhl_df = nhl_df.fillna(0)
corr(nhl_df.iloc[1:, :]);

#### Consice version of Ass-4 Question 2


Same processs for ***NFL*** Datasests (Nation Football League) which is *Rugby* in America

In [192]:
nfl_df = pd.read_csv("assets/nfl.csv")
nfl_df = clean(nfl_df)
nfl_df = giving_reg(nfl_df, df)
nfl_df['reg'] = nfl_df['reg'].replace({"Arizona Cardinals": "Phoenix",
                                "Carolina Panthers": "Charlotte",
                                "New England Patriots": "Boston",
                                 "Oakland Raiders":  "San Francisco Bay Area",
                                "Tennessee Titans" : "Nashville"})

# nfl_df = (df.merge(nfl_df, how="inner", left_on="Metropolitan area", right_on="reg")
#             .loc[:, ["reg", "Population", "W", "L"]])

nfl_df = merger(nfl_df, df)

nfl_df = mk_numeric(nfl_df)

nfl_df = win_loss_ratio(nfl_df)
nfl_df.head()
cor = corr(nfl_df)
print("Correlation: "+ str(cor[0]) + ", P_value: "+ str(cor[1]))

Correlation: 0.004922112149349456, P_value: 0.9797833458363694


#### Consice version of NBA question

In [194]:
nba_df = pd.read_csv("assets/nba.csv")

nba_df = clean(nba_df)
nba_df6 = giving_reg(nba_df, df)
## needed to hard coded some region in assymetry resulted team
nba_df ['reg'] = nba_df['reg'].replace({"Golden State Warriors": "San Francisco Bay Area",
                                "Utah Jazz": "Salt Lake City",
                                "Brooklyn Nets": "New York City"})
nba_df = merger(nba_df, df)
nba_df = mk_numeric(nba_df)
nba_df = win_loss_ratio(nba_df)

#nba_df.head()
nba_cor =corr(nba_df)
print("Correlaiton: "+ str(nba_cor[0])+ ", P_value: "+ str(nba_cor[1]))

Correlaiton: -0.17657160252844611, P_value: 0.3687474160446298


In [196]:
#nba_df[nba_df['team'] == "St. Loius"][["team", "reg"]];

### Consice version of MLB data que

In [197]:
mlb_df = pd.read_csv("assets/mlb.csv")

mlb_df = clean(mlb_df)
mlb_df = giving_reg(mlb_df, df)
# hard coded some region of some team which doesn't have resemble in name!!!
mlb_df['reg'] = mlb_df['reg'].replace({"Oakland Athletics" : "San Francisco Bay Area",
                                    "Texas Rangers"  : "Dallas–Fort Worth",
                                    "Colorado Rockies": "Denver",
                                    "Arizona Diamondbacks" : "Phoenix"})

mlb_df = merger(mlb_df, df)
mlb_df = mk_numeric(mlb_df)
mlb_df = win_loss_ratio(mlb_df)
cor = corr(mlb_df)
print("Correlaiton: "+ str(cor[0])+ ", P_value: "+ str(cor[1]))

Correlaiton: 0.1505230448710485, P_value: 0.4629669779770907


In [198]:
mlb_df[mlb_df['reg'] == "Phoenix"];

In [200]:
[x for x in nhl_df['reg'] if "At" in x];

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

### The null Hypothesis:
**in a given area or region, at least a sport teams in each of sports like `NBA`, `NHL`, `MLB`, `NFL`:**

**H0 = `win_loss_ratio`** of *teams in the respective sports in a given region* **same** in each **sports**


**H1 = `win_loss_ratio`** of *teams in the respective sports in a given region* **different** in each **sports**

In [53]:
# def join_4df1(df1, df2, df3, df4):
#     lis = [df1, df2, df3, df4]
#     # take one df
#     for one in lis:
#         #join with all
#         for others in lis:
            
#             return others
# join_4df(1,2,3,4)


### Joining 4 dataset to each other [ join_4df func]

TO test the significat:
- Each final datasets need to be merged with each other
- total datasets form `4 Dataframe` would be `16 Dataframe`
- for example: NHL-NHL, NHL-NFL, NHL-NBA, NHL-MLB
- total from 4 distinct final datasets, 16 Datasets created 
- inorder to do that, a customized `Function` called `SP_DF_Merge` and ` join-4df` created

In [56]:
def sp_df_merge(df1,df2):
    return df1.merge(df2, how="inner", on="reg")

def join_4df(df1, df2, df3, df4):
    df1_df1 = sp_df_merge(df1, df1)
    df1_df2 = sp_df_merge(df1, df2)
    df1_df3 = sp_df_merge(df1, df3)
    df1_df4 = sp_df_merge(df1, df4)
    
    return (df1_df1, df1_df2, df1_df3, df1_df4)


***An example of Merged Dataset***


In [201]:
nhl_nfl.head()

,reg,Population_x,W_x,L_x,W_L_ratio_x,Population_y,W_y,L_y,W_L_ratio_y
0,Boston,4794447,50.0,20.0,0.714286,4794447,11,5,0.6875
1,Buffalo,1132804,25.0,45.0,0.357143,1132804,6,10,0.3750
2,Chicago,9512999,33.0,39.0,0.458333,9512999,12,4,0.7500
3,Dallas–Fort Worth,7233323,42.0,32.0,0.567568,7233323,10,6,0.6250
4,Denver,2853077,43.0,30.0,0.589041,2853077,6,10,0.3750


### p_value function

in order to get the ***P value of 16 datasets out of 4 datasets***

a function called `p_value(which require giving 4 datasets)`

Giving the P_value of datasets in each!

In [81]:
from scipy.stats import ttest_rel
def p_value(df1, df2, df3, df4):
    pval = []
    for df in [df1, df2, df3, df4]:
        pval.append(ttest_rel(df["W_L_ratio_x"], df["W_L_ratio_y"])[1])
    return pval



In [88]:
nhl_nhl , nhl_nfl, nhl_mlb, nhl_nba = join_4df(nhl_df, nfl_df, mlb_df, nba_df)
nfl_nfl, nfl_nhl, nfl_mlb, nfl_nba = join_4df(nfl_df, nhl_df, mlb_df, nba_df)
mlb_mlb, mlb_nhl, mlb_nfl, mlb_nba = join_4df(mlb_df, nhl_df, nfl_df, nba_df)
nba_nba, nba_nhl, nba_nfl, nba_mlb = join_4df(nba_df, nhl_df, nfl_df, mlb_df)

In [75]:
nhl_nhl.head()

,reg,Population_x,W_x,L_x,W_L_ratio_x,Population_y,W_y,L_y,W_L_ratio_y
0,Atlanta,5789700,0.0,0.0,0.000000,5789700,0.0,0.0,0.000000
1,Boston,4794447,50.0,20.0,0.714286,4794447,50.0,20.0,0.714286
2,Buffalo,1132804,25.0,45.0,0.357143,1132804,25.0,45.0,0.357143
3,Calgary,1392609,37.0,35.0,0.513889,1392609,37.0,35.0,0.513889
4,Chicago,9512999,33.0,39.0,0.458333,9512999,33.0,39.0,0.458333


In [205]:
pval_df = pd.DataFrame({"NHL": p_value(nhl_nhl, nhl_nfl, nhl_mlb, nhl_nba),
                       "NFL": p_value(nfl_nhl, nfl_nfl, nfl_mlb, nfl_nba),
                       "MLB": p_value(mlb_nhl, mlb_nfl, mlb_mlb, mlb_nba),
                       "NBA" : p_value(nba_nhl, nba_nfl, nba_mlb, nba_nba)},
                      index=["NHL", "NFL", "MLB", "NBA"])
pval_df;

## mistake one
# in nhl df, Atlanta shouldn't be

C:\Users\User\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [202]:
nba_nhl.shape;

In [203]:
nba_nhl;

##### Final output arranged in table formet, so it becomes easy to interpret

In [90]:
# mistake solve, in NHL atlanta column removed
pval_df = pd.DataFrame({"NHL": p_value(nhl_nhl, nhl_nfl, nhl_mlb, nhl_nba),
                       "NFL": p_value(nfl_nhl, nfl_nfl, nfl_mlb, nfl_nba),
                       "MLB": p_value(mlb_nhl, mlb_nfl, mlb_mlb, mlb_nba),
                       "NBA" : p_value(nba_nhl, nba_nfl, nba_mlb, nba_nba)},
                      index=["NHL", "NFL", "MLB", "NBA"])
pval_df

C:\Users\User\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


,NHL,NFL,MLB,NBA
NHL,NaN,0.030959,0.000712,0.022316
NFL,0.030959,NaN,0.802384,0.941792
MLB,0.000712,0.802384,NaN,0.951046
NBA,0.022316,0.941792,0.951046,NaN


in **5% alpha or singnificant level**:
We can say, in each given region, the teams in 4 sports have the ***same*** **win loss ratio** or ***different***

The forgone conclusion is

in each region: ***NHL and NFL*** *the wining ratio* the teams in these sport differ significantly

in the same way: ***MLB and NHL*** **differ** significantly, even in ***1% alpha level or significant level***

teams in ***NBA and NHL*** also **differ** significantly

But in context of other teams with the others for instance ***NFL and MLB or MLB and NBA,*** we ***don't have significant evidence*** that those teams' *win loss ration* ****differ**** much 